<a href="https://colab.research.google.com/github/TernFolbaek/angryGPT/blob/googleColab/angryGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/TernFolbaek/angryGPT.git

fatal: destination path 'angryGPT' already exists and is not an empty directory.


In [3]:
!pip install pandas scikit-learn matplotlib convokit transformers 
!ls
!git lfs install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
angryGPT  sample_data
Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.


In [4]:
%cd angryGPT
!pwd

/content/angryGPT
/content/angryGPT


In [5]:
!git remote show origin
!git lfs pull

* remote origin
  Fetch URL: https://github.com/TernFolbaek/angryGPT.git
  Push  URL: https://github.com/TernFolbaek/angryGPT.git
  HEAD branch: main
  Remote branches:
    googleColab tracked
    main        tracked
  Local branch configured for 'git pull':
    main merges with remote main
  Local ref configured for 'git push':
    main pushes to main (up to date)


In [6]:
import os

if os.path.exists('utterances.jsonl'):
    print("utterances.jsonl exists")
else:
    print("utterances.jsonl does not exist")

utterances.jsonl exists


In [7]:
!pip install huggingface_hub
!huggingface-cli login

import pandas as pd
import sklearn
import string
import numpy as np
import matplotlib.pyplot as plt
import pickle
from convokit import Corpus, download
from pandas import json_normalize
import json
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import (GPT2LMHeadModel, GPT2Tokenizer, GPT2Config,
                          DataCollatorForLanguageModeling, Trainer, TrainingArguments)

# Load the corpus
corpus = Corpus(filename=download("conversations-gone-awry-corpus"))
corpus.print_summary_stats()

# Load JSON data and convert to DataFrame
jsonl_file = 'utterances.jsonl'
with open('utterances.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]
flat_data = json_normalize(data)
df = pd.DataFrame(flat_data)
columns_to_drop = [column for column in df.columns if column != 'text']
df = df.drop(columns=columns_to_drop)

flat_data = json_normalize(data)
df = pd.DataFrame(flat_data)
columns_to_drop = [column for column in df.columns if column != 'text']
df = df.drop(columns=columns_to_drop)

# Save DataFrame to CSV file
csv_file = 'provoking_file.csv'
df.to_csv(csv_file, index=False)

# Function to remove punctuation and numbers from text
def remove_pun_and_numbers(text):
    text = str(text)
    to_remove = string.punctuation + string.digits
    text = text.lower()
    return text.translate(str.maketrans('', '', to_remove))

# Load model
def load_model():
    with open('model.pkl', 'rb') as file:
        model = pickle.load(file)
    return model

model = load_model()

# Add the new column to the DataFrame
def only_negatives(message):
    result = model.predict([message])
    if result == ['Negative']:
        return message
    else:
        return None

df['text'] = df['text'].apply(only_negatives)
df = df.dropna(subset=['text'])
df = df.reset_index(drop=True)

# Split the dataset
split_index = len(df) // 2
output_df = df.iloc[split_index:].copy()
input_df = df.iloc[:split_index].copy()
output_df.rename(columns={'text': 'output'}, inplace=True)
input_df.rename(columns={'text': 'input'}, inplace=True)
output_df.reset_index(drop=True, inplace=True)
input_df.reset_index(drop=True, inplace=True)
df = pd.concat([input_df, output_df], axis=1)
df.output.apply(remove_pun_and_numbers)
df.input.apply(remove_pun_and_numbers)

from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")



# Load pre-trained model and tokenizer
def from_local_pretrained(directory):
    config = GPT2Config.from_pretrained(os.path.join(directory, "config.json"))
    tokenizer = GPT2Tokenizer.from_pretrained(directory)
    model = GPT2LMHeadModel.from_pretrained(os.path.join(directory, "pytorch_model.bin"), config=config)
    return tokenizer, model

local_distilgpt2_directory = "distilgpt2"
tokenizer, model = from_local_pretrained(local_distilgpt2_directory)

# Prepare the dataset
eos_token = tokenizer.eos_token
df["text"] = df["input"] + eos_token + df["output"]
df["text"].to_csv("processed_dataset.txt", index=False, header=False)

class CustomTextDataset(Dataset):
    def __init__(self, tokenizer, file_path, block_size):
        with open(file_path, "r") as f:
            text = f.read()
        
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.examples = tokenizer(text, return_tensors="pt", truncation=True, padding=False)["input_ids"]
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

# Create a CustomTextDataset and DataCollator
dataset = CustomTextDataset(tokenizer=tokenizer, file_path="processed_dataset.txt", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="chatbot_output",
    overwrite_output_dir=True,
    num_train_epochs= 10,
    per_device_train_batch_size=1,
    save_steps=10_000,
    gradient_accumulation_steps=1,
    save_total_limit=2,
    logging_steps=500,
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

model.save_pretrained("my_model")
model = GPT2LMHeadModel.from_pretrained("my_model")


# Save the model as a pickle file
with open('my_model.pkl', 'wb') as file:
    pickle.dump(model, file)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add tok

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [8]:
input_text = ["race intelligence"]
input_ids = tokenizer(input_text, return_tensors='pt').input_ids
generated_text = model.generate(input_ids, max_length=100, num_return_sequences=1, do_sample=True)
decoded_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
print(decoded_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


race intelligence, that they never saw it before, but it also showed that he knew it before. If he ever worked with Altham on this topic, he had to look at it in different light and remember that the other members of his group had also been on a similar level.
